In [1]:
from flink_gateway_api import Client
from resinkit import AsyncResinkitClient
import asyncio
import logging

from resinkit.flink_session import FlinkSession

# Configure the root logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logging.getLogger("httpcore.http11").setLevel(logging.INFO)
logging.getLogger('httpcore.connection').setLevel(logging.INFO)

import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops
fg_client = Client(base_url="http://localhost:8083", raise_on_unexpected_status=True)

sql_t0 = '''
SELECT * FROM (
    VALUES
        (23, 'Alice Liddell', CAST('2024-12-29 10:30:00' AS TIMESTAMP)),
        (19, 'Bob Smith', CAST('2024-12-28 11:45:00' AS TIMESTAMP)),
        (27, 'Charlie Brown', CAST('2024-12-27 14:15:00' AS TIMESTAMP)),
        (31, 'David Jones', CAST('2024-12-26 16:20:00' AS TIMESTAMP))
) AS t(age, name, created_at);
'''

with FlinkSession(fg_client) as session:
    with session.execute(sql_t0).sync() as operation:
        df = operation.fetch().sync()


2024-12-30 10:28:23,362 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions "HTTP/1.1 200 OK"
2024-12-30 10:28:23,369 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/5e487f94-5d9f-4566-a2e9-f9cc2298ddc7/statements "HTTP/1.1 200 OK"
2024-12-30 10:28:23,374 - httpx - INFO - HTTP Request: GET http://localhost:8083/sessions/5e487f94-5d9f-4566-a2e9-f9cc2298ddc7/operations/6c5d302f-233b-4c50-be0d-7c990d2f17c6/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-30 10:28:23,375 - resinkit.session_utils - DEBUG - Fetch result: {'nextResultUri': '/v2/sessions/5e487f94-5d9f-4566-a2e9-f9cc2298ddc7/operations/6c5d302f-233b-4c50-be0d-7c990d2f17c6/result/0?rowFormat=JSON', 'resultType': 'NOT_READY'}
2024-12-30 10:28:23,481 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/5e487f94-5d9f-4566-a2e9-f9cc2298ddc7/operations/6c5d302f-233b-4c50-be0d-7c990d2f17c6/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-30 10:28:23,483 - resinkit.session_utils - DEB

In [2]:
df

,age,name,created_at
0,23,Alice Liddell,2024-12-29T10:30:00
1,19,Bob Smith,2024-12-28T11:45:00
2,27,Charlie Brown,2024-12-27T14:15:00
3,31,David Jones,2024-12-26T16:20:00


In [4]:
from resinkit.flink_operation import ResultsFetchOpts

# languange=sql
sql1 = '''
CREATE TABLE mydatabase_User (
  `event_time` TIMESTAMP_LTZ(3) METADATA FROM 'value.source.timestamp' VIRTUAL,  -- from Debezium format
  `origin_table` STRING METADATA FROM 'value.source.table' VIRTUAL, -- from Debezium format
  `partition_id` BIGINT METADATA FROM 'partition' VIRTUAL,  -- from Kafka connector
  `offset` BIGINT METADATA VIRTUAL,  -- from Kafka connector
  `id` STRING,
  `name` STRING,
  `email` STRING,
  `email_verified` TIMESTAMP_LTZ(3),
  `image` STRING,
  `invalid_login_attempts` INT,
  `locked_at` TIMESTAMP_LTZ(3),
  `password` STRING,
  `created_at` TIMESTAMP_LTZ(3),
  `updated_at` TIMESTAMP_LTZ(3)
) WITH (
    'connector' = 'kafka',
    'topic' = 'mydatabase.User',
    'properties.bootstrap.servers' = 'localhost:9092',
    'properties.group.id' = 'mydatabase_user_consumer_group',
    'scan.startup.mode' = 'earliest-offset',
    'format' = 'debezium-json'
);
'''
sql2 = """ ADD JAR '/opt/flink/lib/flink-sql-connector-kafka-3.3.0-1.19.jar' """
sql3 = 'select * from mydatabase_User;'

with FlinkSession(fg_client) as session:
    with session.execute_all([sql1, sql2, sql3]).sync() as operation:
        dfs = operation.fetch_all(fetch_opts=ResultsFetchOpts())

2024-12-30 10:36:07,241 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions "HTTP/1.1 200 OK"
2024-12-30 10:36:07,245 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/96bdaee7-11c7-4430-9810-8b83aeea6dff/statements "HTTP/1.1 200 OK"
2024-12-30 10:36:07,283 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/96bdaee7-11c7-4430-9810-8b83aeea6dff/statements "HTTP/1.1 200 OK"
2024-12-30 10:36:07,296 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/96bdaee7-11c7-4430-9810-8b83aeea6dff/statements "HTTP/1.1 200 OK"
2024-12-30 10:36:07,306 - httpx - INFO - HTTP Request: GET http://localhost:8083/sessions/96bdaee7-11c7-4430-9810-8b83aeea6dff/operations/6258f9ea-6001-4b4b-a731-58307391a6a3/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-30 10:36:07,307 - resinkit.session_utils - DEBUG - Fetch result: {'isQueryResult': False, 'nextResultUri': '/v2/sessions/96bdaee7-11c7-4430-9810-8b83aeea6dff/operations/6258f9ea-6001-4b4b-a731-58

In [7]:
dfs[0]

,result
0,OK


In [8]:
dfs[1]

,result
0,OK


In [9]:
dfs[2]

,event_time,origin_table,partition_id,offset,id,name,email,email_verified,image,invalid_login_attempts,locked_at,password,created_at,updated_at
0,None,User,0,0,b366f235-7991-4fdf-a8fe-600843b6ebf9,Paula Chavez,ortizpaul@example.org,None,https://dummyimage.com/970x335,4,None,84sp7%Uj&f,None,None
1,None,User,0,1,76720e39-8bd1-45d1-9568-ff0923ddd9e8,Amber Walker MD,angelaanderson@example.org,None,https://picsum.photos/1024/315,5,None,#cLDmURJ@3,None,None
2,None,User,0,2,699522bc-4487-41cc-9752-0167d0f30800,Kelsey Mata,gary36@example.com,None,https://placekitten.com/787/400,2,None,sTTQ6FiD)2,None,None
3,None,User,0,3,9679da90-7062-4dba-8451-c8839ce571ab,James Miller,hannah17@example.org,None,https://picsum.photos/595/975,3,None,B(K*14HqJx,None,None
4,None,User,0,4,b52e2329-81a4-4075-bac6-0308216bbaa2,Rebecca Lee,sellersglenn@example.com,None,https://dummyimage.com/965x118,3,None,&c04uDtS3#,None,None
5,None,User,0,5,a1281172-395a-4975-b21d-6845a7c40cdb,Nicole Hall,martinsarah@example.net,None,https://picsum.photos/94/337,2,None,&mX%TadJ9Q,None,None
6,None,User,0,6,9d2c088d-befe-4336-ad86-0f8ce6707b2a,Julie Morrison,ngraham@example.com,None,https://placekitten.com/1001/358,2,None,0S!X^$3o#7,None,None
7,None,User,0,7,1a0ee505-0ae7-40b5-9372-6a58f31cb844,Anita Collins,stevenpearson@example.org,None,https://picsum.photos/584/401,2,None,499WptMb#D,None,None
8,None,User,0,8,d535b7be-49cc-446f-9079-0401e6c14916,Theresa Howard,michaelcampbell@example.com,None,https://placekitten.com/73/646,3,None,Bi15!ZVm$4,None,None
9,None,User,0,9,b5df828a-7649-4fef-a2ea-3abe40aaea65,Richard Salas,reedlaurie@example.com,None,https://dummyimage.com/315x425,5,None,s$A4q5In_&,None,None
